In [53]:
import pandas as pd
import numpy as np
import os
import ast

from datetime import datetime

CARGA DE DATA

In [54]:
df_labs = pd.read_csv("Data\movies_dataset.csv")
df_labs.head()

C:\Users\pecor\AppData\Local\Temp\ipykernel_21356\180640151.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_labs = pd.read_csv("Data\movies_dataset.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [55]:
df_labs.info()
df_labs.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

(45466, 24)

01.ELIMINAR COLUMNAS

In [56]:
df_labs.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage', 'tagline'], axis=1, inplace = True)

02.VALORES NULOS REVENUE Y BUDGET

In [57]:
df_labs['budget'] = pd.to_numeric(df_labs['budget'], errors='coerce').fillna(0)

In [58]:
df_labs['revenue'] = df_labs['revenue'].fillna(0)

03.Crear Columna Return

In [59]:
df_labs['return'] = df_labs.apply(lambda x: x['revenue']/x['budget'] if x['budget'] != 0 else 0, axis=1)

In [60]:
df_labs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4494 non-null   object 
 1   budget                 45466 non-null  float64
 2   genres                 45466 non-null  object 
 3   id                     45466 non-null  object 
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45461 non-null  object 
 7   production_companies   45463 non-null  object 
 8   production_countries   45463 non-null  object 
 9   release_date           45379 non-null  object 
 10  revenue                45466 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       45460 non-null  object 
 13  status                 45379 non-null  object 
 14  title                  45460 non-null  object 
 15  vo

04. DESANIDAR 

04.1 String a Diccionario

In [61]:
def str_to_dict(value):
    """Convert string a diccionario

    Args:
        value (str): String valu se convierte en diccionario

    Returns:
        None : Return None if NaN value is present
    """
    if pd.isna(value):
        return None
    else:
        return ast.literal_eval(value)

04.2 Desanidar Columnas

In [62]:
df_labs['belongs_to_collection'] = df_labs['belongs_to_collection'].apply(str_to_dict)

In [63]:
df_nested = pd.json_normalize(df_labs['belongs_to_collection']).fillna('')
df_nested.head()

,id,name,poster_path,backdrop_path
0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,,,,
2,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3,,,,
4,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg


In [64]:
columns = df_nested.columns
new_columns_names = {column : f'belongs_to_collections_{column}' for column in columns}
df_nested.rename(columns=new_columns_names, inplace=True)
df_nested.head()

,belongs_to_collections_id,belongs_to_collections_name,belongs_to_collections_poster_path,belongs_to_collections_backdrop_path
0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,,,,
2,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3,,,,
4,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg


In [65]:
df_nested.shape

(45466, 4)

In [66]:
columns = ['genres', 'production_companies', 'production_countries', 'spoken_languages']

for column in columns:
    df_labs[column] = df_labs[column].apply(str_to_dict)
    df_nested = pd.json_normalize(df_labs[column]).fillna('')
    nested_columns = df_nested.columns
    df_nested_1 = pd.DataFrame()
    for nested_column in nested_columns:
        df_aux = pd.json_normalize(df_nested[nested_column]).fillna('')
        if 'production_countries' in column:
            df_aux.rename(columns={'iso_3166_1': 'id'}, inplace=True)
        if 'spoken_languages' in column:
            df_aux.rename(columns={'iso_639_1': 'id'}, inplace=True)
        df_aux.drop(columns='id', axis=1, inplace=True)
        new_columns_names = {col : f'{column}_{nested_column}_{col}' for col in df_aux.columns}
        df_aux.rename(columns=new_columns_names, inplace=True)
        df_nested_1 = pd.concat([df_nested_1, df_aux], axis=1)

    # Convertir columns to a list of columns
    column_name = f'{column}_name'
    column_name_list = df_nested_1.columns.to_list()
    df_nested_1[column_name] = df_nested_1.values.tolist()
    df_nested_1.drop(columns=column_name_list, axis=1, inplace=True)
    df_labs = pd.concat([df_labs, df_nested_1], axis=1)
    df_labs.drop(columns=column, inplace=True)

In [67]:
df_labs.shape

(45466, 18)

In [68]:
df_labs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   belongs_to_collection      4494 non-null   object 
 1   budget                     45466 non-null  float64
 2   id                         45466 non-null  object 
 3   original_language          45455 non-null  object 
 4   overview                   44512 non-null  object 
 5   popularity                 45461 non-null  object 
 6   release_date               45379 non-null  object 
 7   revenue                    45466 non-null  float64
 8   runtime                    45203 non-null  float64
 9   status                     45379 non-null  object 
 10  title                      45460 non-null  object 
 11  vote_average               45460 non-null  float64
 12  vote_count                 45460 non-null  float64
 13  return                     45466 non-null  flo

05. VALORES NULOS RELEASE_DATE

In [69]:
df_labs.dropna(subset=['release_date'], inplace=True)
df_labs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45379 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   belongs_to_collection      4491 non-null   object 
 1   budget                     45379 non-null  float64
 2   id                         45379 non-null  object 
 3   original_language          45368 non-null  object 
 4   overview                   44438 non-null  object 
 5   popularity                 45377 non-null  object 
 6   release_date               45379 non-null  object 
 7   revenue                    45379 non-null  float64
 8   runtime                    45130 non-null  float64
 9   status                     45296 non-null  object 
 10  title                      45376 non-null  object 
 11  vote_average               45376 non-null  float64
 12  vote_count                 45376 non-null  float64
 13  return                     45379 non-null  flo

06. FORMATO FECHA

06.1 CREAR COLUMNA RELEASE YEAR

In [70]:
regex_filter = '^\d{4}\-(0?[1-9]|1[012])\-(0?[1-9]|[12][0-9]|3[01])$'
filter = df_labs['release_date'].str.contains(regex_filter)
df_labs = df_labs[filter]
df_labs['release_year'] = df_labs['release_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date().year)

C:\Users\pecor\AppData\Local\Temp\ipykernel_21356\312493890.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filter = df_labs['release_date'].str.contains(regex_filter)


06.2 CONVERTIR EL RELEASE DATE A FORMATO FECHA

In [71]:
df_labs['release_date'] = pd.to_datetime(df_labs['release_date'], format='%Y-%m-%d')

In [74]:
print(type(df_labs))

<class 'pandas.core.frame.DataFrame'>


MODULO API

In [76]:
from fastapi import FastAPI
from fastapi.responses import JSONResponse
import pandas as pd

app = FastAPI()

@app.get("/dataframe")
def get_dataframe():
    data = df_labs
    return JSONResponse(content=data.to_json(orient='records'), media_type='application/json')

import nest_asyncio
nest_asyncio.apply()
import uvicorn

if __name__ == '__main__':
    uvicorn.run(app, host='0.0.0.0', port=8000)

INFO:     Started server process [21356]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:60814 - "GET /dataframe HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [21356]
